In [61]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader, TensorDataset
import numpy as np
import scipy.io as sio
from torch.utils.data import TensorDataset
import pickle

In [62]:
with open("/Users/lkk/Documents/BOUN CMPE/CMPE 537-Computer Vision/Term Project/Code/Datasets/CUB_Paper/produced/image_tensors_256.pkl", 'rb') as f:
    image_tensors = pickle.load(f)

In [63]:
image_tensors = torch.tensor(image_tensors)

In [64]:
from torchvision.models import ViT_B_16_Weights

In [65]:
vit_transform = ViT_B_16_Weights.DEFAULT.transforms()

In [66]:
vit_images = []
for image in image_tensors:
    trans_image = vit_transform(image)
    vit_images.append(trans_image)

In [67]:
vit_images = torch.stack(vit_images, dim=0)

In [68]:
vit_dataset = TensorDataset(vit_images)
vit_dataloader = DataLoader(vit_dataset, batch_size=32, shuffle=False)

In [69]:
vit_model = models.vit_b_16(weights=ViT_B_16_Weights.DEFAULT)
vit_feature_extractor = torch.nn.Sequential(*list(vit_model.children())[:-1])
conv = vit_feature_extractor[0]
encoder = vit_feature_extractor[1]

In [70]:
vit_model.eval()
vit_feature_extractor.eval()
conv.eval()
encoder.eval()

Encoder(
  (dropout): Dropout(p=0.0, inplace=False)
  (layers): Sequential(
    (encoder_layer_0): EncoderBlock(
      (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (self_attention): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
      )
      (dropout): Dropout(p=0.0, inplace=False)
      (ln_2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): MLPBlock(
        (0): Linear(in_features=768, out_features=3072, bias=True)
        (1): GELU(approximate='none')
        (2): Dropout(p=0.0, inplace=False)
        (3): Linear(in_features=3072, out_features=768, bias=True)
        (4): Dropout(p=0.0, inplace=False)
      )
    )
    (encoder_layer_1): EncoderBlock(
      (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (self_attention): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
      )
    

In [71]:
vit_features = []
with torch.no_grad():
    for batch in vit_dataloader:
        image_batch = batch[0]
        x = vit_model._process_input(image_batch)
        n = x.shape[0]
        batch_class_token = vit_model.class_token.expand(n, -1, -1)
        x = torch.cat([batch_class_token, x], dim=1)
        x = encoder(x)
        batch_features = x[:, 0]
        vit_features.append(batch_features)

In [76]:
vit_features = torch.cat(vit_features, dim=0)

In [80]:
with open("/Users/lkk/Documents/BOUN CMPE/CMPE 537-Computer Vision/Term Project/Code/Datasets/CUB_Paper/produced/vit_features.pkl", 'wb') as f:
    pickle.dump(vit_features, f)